<a href="https://colab.research.google.com/github/alexcarvajal/ModeloPrediccionBRT/blob/main/Optimizaci%C3%B3nHiperpar%C3%A1metros/OptunaRF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.4 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import optuna
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import psutil
import time
start_time = time.time()

# Carga de datos
url1 = '/content/drive/MyDrive/Reuniones/DatasetsWP2/Dataset1CodificadoV2.csv'
data1 = pd.read_csv(url1)
data1.drop(['Unnamed: 0'], axis=1, inplace=True)
X = data1.drop(columns=['Inputs'])
y = data1['Inputs']

def objective(trial):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    n_estimators = trial.suggest_int('n_estimators', 10, 1000)
    max_depth = trial.suggest_int('max_depth', 2, 32, log=True)
    min_samples_split = trial.suggest_float('min_samples_split', 0.1, 1.0)
    min_samples_leaf = trial.suggest_float('min_samples_leaf', 0.1, 0.5)


    model = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )
    model.fit(X_train, y_train)

    # Predicciones y evaluación
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)

    return rmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# Imprimir los mejores hiperparámetros
print('Mejores hiperparámetros:', study.best_trial.params)
print('Mejor resultado:', study.best_value)


process = psutil.Process()
print(f"Memory usage: {process.memory_info().rss / (1024 * 1024)} megabytes")

end_time = time.time()
execution_time = end_time - start_time

# Calcular minutos y segundos
minutes = int(execution_time // 60)
seconds = int(execution_time % 60)

# Formatear y mostrar el tiempo de ejecución
print(f"Tiempo de ejecución: {minutes:02d}:{seconds:02d}")



[I 2024-04-01 14:06:50,028] A new study created in memory with name: no-name-0eef89c6-3f42-4215-80ec-85135facc796
[I 2024-04-01 14:07:01,418] Trial 0 finished with value: 31.273567182935604 and parameters: {'n_estimators': 133, 'max_depth': 2, 'min_samples_split': 0.2948639223058845, 'min_samples_leaf': 0.31179452931522544}. Best is trial 0 with value: 31.273567182935604.
[I 2024-04-01 14:07:09,771] Trial 1 finished with value: 31.545625296652755 and parameters: {'n_estimators': 508, 'max_depth': 12, 'min_samples_split': 0.12255235105172216, 'min_samples_leaf': 0.3350332660013212}. Best is trial 0 with value: 31.273567182935604.
[I 2024-04-01 14:07:54,014] Trial 2 finished with value: 29.53378015343369 and parameters: {'n_estimators': 196, 'max_depth': 19, 'min_samples_split': 0.19522792394977756, 'min_samples_leaf': 0.10500877012567784}. Best is trial 2 with value: 29.53378015343369.
[I 2024-04-01 14:08:06,942] Trial 3 finished with value: 31.545627652735554 and parameters: {'n_estima

Mejores hiperparámetros: {'n_estimators': 144, 'max_depth': 6, 'min_samples_split': 0.1516617336235904, 'min_samples_leaf': 0.10015065113480498}
Mejor resultado: 29.46960940466909
Memory usage: 362.1796875 megabytes
Tiempo de ejecución: 86:04
